In [7]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [8]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [10]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [11]:
# reflect an existing database into a new model

base = automap_base()


# reflect the tables

base.prepare(engine, reflect=True)

# inspector = inspect(engine)

# inspector.get_table_names()

In [12]:
# View all of the classes that automap found

base.classes.keys()

['measurement', 'station']

In [13]:
# Save references to each table
# Demographics = Base.classes.demographics

Station = base.classes.station

Measurement = base.classes.measurement

In [14]:
# Create our session (link) from Python to the DB

session = Session(engine)


# Exploratory Precipitation Analysis

In [24]:
# Find the most recent date in the data set.

initial_date = session.query(Measurement.date).order_by(Measurement.date).first()

query_date = dt.date(2010, 1, 1) + dt.timedelta(days=365)

print(initial_date)
print(query_date)


('2010-01-01',)
2011-01-01


In [26]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Note you do not pass in the date as a variable to your query.




precip_data = session.query(func.strftime("%Y-%m-%d", Measurement.date), Measurement.prcp).\
    filter(func.strftime("%Y-%m-%d", Measurement.date) <= query_date).all()




# Save the query results as a Pandas DataFrame and set the index to the date column


precip_df = pd.DataFrame(precip_data, columns = ['date', 'prcp'])

precip_df.set_index('date', inplace=True)

precip_df




# Sort the dataframe by date


precip_df = precip_df.sort_values(by='date')
precip_df.head()


    # session.query(Dow.date).\
    #     filter(Dow.date > '2011-03-01').\
    #     order_by(Dow.date).all()

    # session.query(Dow.date, Dow.close_price).\
    #     filter(Dow.stock == 'CSCO').\
    #     filter(Dow.date == query_date).all()
    # query_date = dt.date(2011, 4, 8) - dt.timedelta(days=7)
    # print("Query Date: ", query_date)
    
    






    # sel = [Measurement.date, Measurement.prcp,
    #         func.sum(prcp)]
    # precip_data = session.query(*sel).\
    #     filter(Measurement.date < '2011-01-01').\
    #     group_by(Measurement.date).\
    #     order_by(Measurement.date).all()
    # precip_data




            # sel = [Dow.stock, 
            #        func.avg(Dow.open_price), 
            #        func.avg(Dow.high_price), 
            #        func.avg(Dow.low_price), 
            #        func.avg(Dow.close_price)]
            # may_averages = session.query(*sel).\
            #     filter(func.strftime("%m", Dow.date) == "05").\
            #     group_by(Dow.stock).\
            #     order_by(Dow.stock).all()
            # may_averages




# Use Pandas Plotting with Matplotlib to plot the data

        # df = pd.DataFrame(may_averages, columns=['stock', 'open_avg', 'high_avg', 'low_avg', 'close_avg'])
        # df.set_index('stock', inplace=True)
        # df.plot.bar()
        # plt.tight_layout()
        # plt.show()













,prcp
date,
2010-01-01,0.08
2010-01-01,0.21
2010-01-01,0.15
2010-01-01,0.15
2010-01-01,0.05


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()